In [2]:
import pandas as pd
import numpy as np
import re
import html
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

# 1. Caricamento Dati
df = pd.read_csv('winter_project_2026/development.csv')

# 2. Preprocessing e Pulizia
def clean_text(text):
    if pd.isna(text):
        return ""
    # Decodifica entità HTML (es. &amp; -> &)
    text = html.unescape(text)
    # Rimozione tag HTML
    text = re.sub(r'<[^>]+>', ' ', text)
    # Rimozione caratteri speciali e numeri (manteniamo solo lettere)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    # Rimozione spazi multipli e lowercase
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

print("Inizio pulizia del testo...")
# Uniamo titolo e articolo per avere più informazione
df['text_raw'] = df['title'].fillna('') + " " + df['article'].fillna('')
df['cleaned_text'] = df['text_raw'].apply(clean_text)
print("Pulizia completata.")

# 3. Split Train/Validation
X = df['cleaned_text']
y = df['label']

# Stratify è fondamentale per il bilanciamento delle classi
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Dimensioni Train: {X_train.shape}, Dimensioni Validation: {X_val.shape}")

# 4. Vectorization (TF-IDF)
# Limitiamo a 10000 feature per velocità e memoria, rimuoviamo stop words inglesi
tfidf = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1, 1))

print("Vettorizzazione in corso...")
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
print("Vettorizzazione completata.")

# 5. Training Modello Baseline (Logistic Regression)
# class_weight='balanced' è cruciale per la metrica Macro F1 su dataset sbilanciati
model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42)

print("Addestramento modello...")
model.fit(X_train_tfidf, y_train)

# 6. Valutazione
y_pred = model.predict(X_val_tfidf)

macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"\n--- Risultati Baseline ---")
print(f"Macro F1 Score: {macro_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

Inizio pulizia del testo...
Pulizia completata.
Dimensioni Train: (63997,), Dimensioni Validation: (16000,)
Vettorizzazione in corso...
Vettorizzazione completata.
Addestramento modello...

--- Risultati Baseline ---
Macro F1 Score: 0.6342

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.66      0.67      4709
           1       0.66      0.75      0.70      2118
           2       0.69      0.74      0.71      2232
           3       0.57      0.47      0.51      1995
           4       0.74      0.92      0.82      1715
           5       0.44      0.34      0.38      2611
           6       0.53      0.80      0.64       620

    accuracy                           0.64     16000
   macro avg       0.62      0.67      0.63     16000
weighted avg       0.63      0.64      0.63     16000

